In [11]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

with open('/Users/jieyizhang/Downloads/yelp_dataset/yelp_academic_dataset_review.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

review_df = pd.DataFrame(data)

In [12]:
review_df.dropna(inplace=True)
review_df[review_df['stars'] != 3]
review_df['Positivity'] = np.where(review_df['stars'] > 3, 1, 0)
review_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,Positivity
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g,0
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g,1
2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g,0
3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g,0
4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g,1


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(review_df['text'], review_df['Positivity'], random_state = 0)
print('X_train first entry: \n\n', X_train[0])
print('\n\nX_train shape: ', X_train.shape)

# Looking at X_train, we can see that we have a collection of 4497747 reviews or documents.

X_train first entry: 

 The pizza was okay. Not the best I've had. I prefer Biaggio's on Flamingo / Fort Apache. The chef there can make a MUCH better NY style pizza. The pizzeria @ Cosmo was over priced for the quality and lack of personality in the food. Biaggio's is a much better pick if youre going for italian - family owned, home made recipes, people that actually CARE if you like their food. You dont get that at a pizzeria in a casino. I dont care what you say...


X_train shape:  (4497747,)


In [14]:
from sklearn.feature_extraction.text import HashingVectorizer

vect = HashingVectorizer( ngram_range = (1,2)).fit(X_train)
X_train_vectorized = vect.transform(X_train)

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))

In [16]:
from sklearn.metrics import roc_auc_score

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.9139750398277859


In [17]:
sorted_coef_index = model.coef_[0].argsort()
print('Smallest Coef: \n{}\n'.format(sorted_coef_index[:10]))
print('Largest Coef: \n{}\n'.format(sorted_coef_index[:-11:-1]))

Smallest Coef: 
[360204 917640 980514 389865 259686 212338 603374  88781 854154 223976]

Largest Coef: 
[ 65182 986238 900899 625870 377192 647010   2590  94110 276560    270]



In [35]:
print(model.predict(vect.transform(['The food is healthy!',\
                                    'The food was cold.',\
                                    'We like the environment.',\
                                    'The fish was not so good.'])))

[1 0 1 0]
